## Connect

In [7]:
import os, sys
sys.path.append(os.path.join('..', 'osiris'))
from osiris_global import set_runtime_env
set_runtime_env(debug = True, interactive_nb=True)

In [2]:
import os, sys
import graphistry
graphistry.register(api=3, username=os.environ['GRAPHISTRY_USER'], password=os.environ['GRAPHISTRY_PASS'], protocol='https', server='hub.graphistry.com')

Starting new HTTPS connection (1): hub.graphistry.com:443
https://hub.graphistry.com:443 "POST /api-token-auth/ HTTP/1.1" 201 551
JWT refresh via creds
Starting new HTTPS connection (1): hub.graphistry.com:443
https://hub.graphistry.com:443 "POST /api-token-auth/ HTTP/1.1" 201 551


In [10]:
g = graphistry.tigergraph(
    protocol='https', server='osiris-dev2.i.tgcloud.io',
    user=os.environ['OSIRIS_GRAPH_SERVER_USER'], pwd=os.environ['OSIRIS_GRAPH_SERVER_PASS'], 
    db='GDELT_Events'
)

## Query and Visualize

In [11]:
g3 = g.gsql("""
  INTERPRET QUERY () FOR GRAPH GDELT_Events { 
  ListAccum<EDGE> @edgeList;
  actors = 
    select s from Actor:s-(Acted>:a)-Event:t
    where t.event_date > "2022-01-01"
    accum s.@edgeList += a
    limit 100;
  }
    """, 
  {'edges': 'actors.attributes.@edgeList', 'nodes': 'actors'}
)       
g3.plot()

#g3.plot()

Starting new HTTPS connection (1): osiris-dev2.i.tgcloud.io:14240
https://osiris-dev2.i.tgcloud.io:14240 "POST /gsqlserver/interpreted_query HTTP/1.1" 200 None


c:\python\venv\osiris\lib\site-packages\graphistry\tigeristry.py:135: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nodes_df = pd.DataFrame({'node_id': edges_df['from_id'].append(edges_df['to_id'])}) \


KeyError: "['from_type'] not in index"